In [2]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()
sc =SparkContext()
sqlCtx = SQLContext(sc)

In [3]:
rdd = sc.textFile("gs://ds-url-catag/Email_requests_from_agnes/Request_6/request_5.csv")

In [4]:
header = rdd.first()

In [5]:
emails_before = rdd.filter(lambda line: line != header).map(lambda line: line.split(",")).toDF(['email'])

repart_emails = emails_before.repartition(9)

In [6]:
emails_before.count()

3686885

In [38]:
repart_emails.write.mode('overwrite').text("gs://ds-url-catag/Email_requests_from_agnes/Request_6/text_format/")

In [26]:
#processing the rest of emails
final_set = sc.textFile("gs://ds-url-catag/Email_requests_from_agnes/Request_5/first_set/")

In [27]:
head = final_set.first()

In [28]:
final_set.first()

u'"InputData","ResultStatus","ResultEmailAddress","ResultEmailAddressLocalPart","ResultEmailAddressDomainPart","ResultAsciiEmailAddressDomainPart","ResultHasInternationalDomainName","ResultHasInternationalMailboxName","ResultIsCatchAllFailure","ResultIsDisposableEmailAddress","ResultIsRoleAccount","ResultIsDnsFailure","ResultIsMailboxFailure","ResultIsNetworkFailure","ResultIsSmtpFailure","ResultIsSuccess","ResultIsSyntaxFailure","ResultIsTimeoutFailure","ResultSyntaxFailureIndex","CreatedOn","CompletedOn"'

In [29]:
final_set.count()

3554176

In [30]:
writing_emails = final_set.filter(lambda line: line != head).map(lambda line: line.split(",")).toDF(["InputData","ResultStatus","ResultEmailAddress","ResultEmailAddressLocalPart","ResultEmailAddressDomainPart","ResultAsciiEmailAddressDomainPart","ResultHasInternationalDomainName","ResultHasInternationalMailboxName","ResultIsCatchAllFailure","ResultIsDisposableEmailAddress","ResultIsRoleAccount","ResultIsDnsFailure","ResultIsMailboxFailure","ResultIsNetworkFailure","ResultIsSmtpFailure","ResultIsSuccess","ResultIsSyntaxFailure","ResultIsTimeoutFailure","ResultSyntaxFailureIndex","CreatedOn","CompletedOn"])

In [31]:
writing_emails.select(["InputData","ResultStatus"]).where(col('ResultStatus') == '"Success"').count()

2356311

In [32]:
# status_success = 
result_success = writing_emails.select(["InputData","ResultStatus"]).where(col('ResultStatus') == '"Success"')
result_success = result_success.withColumn("InputData",regexp_replace(col('InputData'),'"',''))

In [33]:
result_success.show()

+--------------------+------------+
|           InputData|ResultStatus|
+--------------------+------------+
|changemylife@gmai...|   "Success"|
|arjiputra100@gmai...|   "Success"|
|ilyasa.yazha@gmai...|   "Success"|
|laura.kertapawita...|   "Success"|
|putra.sunda332@gm...|   "Success"|
|astri.yuliawati1@...|   "Success"|
|vixionerherdi@gma...|   "Success"|
|kenizzunlazuardy@...|   "Success"|
|afriguswandi.94@g...|   "Success"|
|  deaobita@gmail.com|   "Success"|
| halimr073@gmail.com|   "Success"|
| pardekptk@gmail.com|   "Success"|
|ekaputrautomo@gma...|   "Success"|
|atikah.herlina88@...|   "Success"|
|miftasayangendo@g...|   "Success"|
|novandryekonugroh...|   "Success"|
| ali148231@gmail.com|   "Success"|
|ajanono011@gmail.com|   "Success"|
| varhan039@gmail.com|   "Success"|
|nurfitrianto717@g...|   "Success"|
+--------------------+------------+
only showing top 20 rows



In [34]:
result_success.select(["InputData"]).coalesce(1).write.mode('overwrite').format("com.databricks.spark.csv").save("gs://ds-url-catag/Email_requests_from_agnes/Request_5/final_results/")   

In [35]:
check = sc.textFile("gs://ds-url-catag/Email_requests_from_agnes/Request_5/final_results/")

In [36]:
check.count()

2356311

In [37]:
check.take(10)

[u'changemylife@gmail.com',
 u'arjiputra100@gmail.com',
 u'ilyasa.yazha@gmail.com',
 u'laura.kertapawitan20@gmail.com',
 u'putra.sunda332@gmail.com',
 u'astri.yuliawati1@gmail.com',
 u'vixionerherdi@gmail.com',
 u'kenizzunlazuardy@gmail.com',
 u'afriguswandi.94@gmail.com',
 u'deaobita@gmail.com']